## Qatar University Grandstand Simulator (QUGS) Damage Detection using Conv3D and MTL

In this notebook, we present the implementation of a 3D Convolutional Neural Network (CNN) and multi-task learning approach on the Qatar University Grandstand Simulator (QUGS) dataset to detect structural damages.

The QUGS dataset represents a structural system with 30 joints, as illustrated in the image below:
![QUGS Joints](http://www.structuralvibration.com/img/i2.png)

Each joint is equipped with accelerometers, and damages are induced by loosening the bolts, as depicted in the image below:
![Damage Application](http://www.structuralvibration.com/img/i3.png)

Our objective in this notebook is to utilize deep learning techniques to detect damages in this structural system using the vibration measurements obtained from the accelerometers. We employ a 3D CNN to capture the spatial and temporal patterns in the data, enabling the accurate identification of structural changes.

For a more comprehensive understanding of the structure and the tests conducted, we recommend referring to the following link:
[QUGS Benchmark](http://www.structuralvibration.com/benchmark/)

Through this notebook, we aim to demonstrate the effectiveness of our proposed approach in detecting structural damages using Deep learning techniques.



#Import

In [ ]:
import gdown
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
import keras
from keras.models import load_model, Model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
import seaborn as sns

In [ ]:
##**************************
##****** elavation**********
##**************************
def plot_performance(hist,epoch, title, save='/content/',Legs=None):
    keyss=list(hist.history.keys())
    if Legs:
      legs=Legs
    else:
      legs=keyss


    colors= ['red','blue','green','#FFA500']


    fig, ax = plt.subplots(figsize=(10, 6))
    for i in range(0,len(keyss),2):
      histor = np.array(hist.history[keyss[i+1]])
      ax.plot(range(1,epoch+1),histor*100, color=colors[i//2], label=legs[i+1])#

    # Add a legend to the plot
    # ax.legend(loc='right')

    # Set the axis labels and title
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy (%)')
    # ax.set_title('Title')
    ax.set_ylim((0,100.5))
    ax.set_xlim((1,epoch))
    ax.grid(axis='x', color='0.85')
    ax.grid(axis='y', color='0.85')
    ax.set_title(title, fontsize=14, fontweight='bold')

    ax2 = ax.twinx()

    for i in range(0,len(keyss),2):
      histor = hist.history[keyss[i]]
      ax2.plot(range(1,epoch+1),histor, color=colors[i//2],linestyle='--', label=legs[i])#

    # Add a legend to the plot


    # Set the axis labels and title
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss',color='red')
    ax2.tick_params(axis='y', colors='red')

    lines1, labels1 = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    # print(lines1, labels1 ,lines2, labels2 )
    ax2.legend(lines1 + lines2, labels1 + labels2, loc='right')
    plt.savefig(save+'.png', dpi=800)

    # convert the history.history dict to a pandas DataFrame:
    hist_df = pd.DataFrame(hist.history)

    # or save to csv:
    hist_csv_file = save+'-history.csv'
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)

    fig.tight_layout()
    plt.show()
    if save:
      fig.savefig(save+'-'+title+'.jpg',dpi=300)
    plt.show()





#%% Defined Confusion Matrix plot function
def PlotConfusionMatrix(y_test ,y_pred_probs, title='Confusion Matrix', figsize=(8,8), save='/content/'):

    cf_matrix=confusion_matrix(np.argmax(y_test,axis = 1), np.argmax(y_pred_probs,axis = 1))
    # DetaFrame_cm = pdNew.DataFrame(array, range(6), range(5))
    fig = plt.figure(figsize=figsize)
    ax2 = plt.axes()
    ax2.set_title(title)
    ax2.xaxis.tick_top()
    ConfusionMatrixDisplay(cf_matrix).plot(ax=ax2,cmap='Blues',colorbar=False)

    if save:
      fig.savefig(save+'-'+title+'.jpg', dpi=300)
    plt.show()

def uneqCM(confmatdata,fsize=(10,8),title='Confusion Matrix (Multi Task)',save='/content/'):
    fig, ax = plt.subplots(figsize=fsize)
    ax=sns.heatmap(confusion_matrix(confmatdata[:,0], confmatdata[:,1])[:-1],cmap='Blues',annot=True,cbar=False)
    ax.set(xlabel='Predicted Category',ylabel='True category' )
    ax.set_title(title)
    ax.xaxis.tick_top()
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['left'].set_visible(True)
    ax.spines['bottom'].set_visible(True)

    if save:
      fig.savefig(save+'-'+title+'.jpg', dpi=300)
    plt.show()


# claculate model scores
def score_model(y_test, y_pred_oh, y_pred_probs, save ='/content/'):

    acc = accuracy_score(y_test, y_pred_oh)
    prec = precision_score(y_test, y_pred_oh, average = 'weighted')
    rec = recall_score(y_test, y_pred_oh, average = 'weighted')
    fscore = f1_score(np.argmax(y_test,axis = 1), np.argmax(y_pred_oh,axis = 1), average='weighted')
    cmatrix = confusion_matrix(np.argmax(y_test,axis = 1), np.argmax(y_pred_oh,axis = 1))
    # _, conf = pred_confidence(y_pred_probs)

    scores = {'accuracy' : acc,
              'precision' : prec,
              'recall' : rec,
              'F1score': fscore,
              # 'Confidence' : conf,
              'Confusion': cmatrix
              }

    if save:
        import csv
        with open(save+'-Scores','w') as f:
            for key in scores.keys():
                f.write("%s,%s\n"%(key,scores[key]))
    del scores['Confusion']
    return scores

# Calculate task-specific accuracy
def MT_eval(Y_pred,Y_test,num_tasks,save):
  task_acc = []
  MT_Mat = np.zeros((num_tasks, 4))
  for i in range(num_tasks):
      task_y_true = Y_test[:, i]
      task_y_pred = Y_pred[:, i]
      task_acc.append(accuracy_score(task_y_true, task_y_pred))

  # Calculate average accuracy
  avg_acc = np.mean(task_acc)


  # Plot confusion matrices for all tasks
  fig, axes = plt.subplots(nrows=5, ncols=6, figsize=(12, 9))
  for i, ax in enumerate(axes.flatten()):
      if i < num_tasks:
          task_y_true = Y_test[:, i]
          task_y_pred = Y_pred[:, i]
          MT_Mat[i, 0] = accuracy_score(task_y_true, task_y_pred)
          MT_Mat[i, 1] = precision_score(task_y_true, task_y_pred)
          MT_Mat[i, 2] = recall_score(task_y_true, task_y_pred)
          MT_Mat[i, 3] = f1_score(task_y_true, task_y_pred)




          cm = confusion_matrix(task_y_true, task_y_pred)

          if cm.shape == (1, 1):
          # add dummy row and column with zero value
            cm=np.pad(cm,((0, 1), (0, 1)))

          sns.heatmap(cm, annot=True, cmap='Blues', ax=ax, fmt='g',cbar=False)
          ax.set_title('Task ' +str(i+1), fontsize=10)
          ax.set_ylabel('True', fontsize=10)
          ax.set_xlabel('Predicted', fontsize=10)
          # ax.set_xticks([0,1])
          # ax.set_yticks([0,1])
          ax.tick_params(axis='x', labelsize=8)
          ax.tick_params(axis='y', labelsize=8)
          for text in ax.texts:
            text.set_fontsize(8)
      else:
          fig.delaxes(ax)
            # save results to CSV file
  np.savetxt(save+'evaluation_metrics.csv', MT_Mat, delimiter=",")
  plt.tight_layout()
  plt.show()
  fig.savefig(save+'-'+'MT-Eval.png',dpi=500)
  plt.show()

#loading data and basic model constructor

In [ ]:
#######################################################
##################### Map3D ###########################
#######################################################

def Map3D(DataTensor,nrow,TempDimLen,nly,labels=None):
  nclass=DataTensor.shape[0]

  for idx in range(nclass):
      respmat2=DataTensor[idx]
      respmat2=np.array(np.vsplit(respmat2, nrow))
      Xseti=np.array(np.split(respmat2, DataTensor.shape[2]//TempDimLen, axis=-1))
      if not labels:
        Yseti=np.zeros((Xseti.shape[0],nclass))
        Yseti[:,idx]=1
      elif labels:
        Yseti=np.zeros((Xseti.shape[0],nly))
        if idx!=0:
          Yseti[:,[x-1 for x in labels[idx]]]=1

      try:
        Xset=np.vstack((Xset, Xseti))
        Yset=np.vstack((Yset, Yseti))
      except:
        Xset=Xseti
        Yset=Yseti
      del(respmat2)
  return Xset, Yset


#######################################################
################### LoadData ##########################
#######################################################
def DownLoadData(DatasetType='single',mode='cls'):
  Dataset=DatasetType.lower()
  try:
    os.remove('/content/SingleA.npy')
    os.remove('/content/SingleB.npy')

  except:
    pass
  url1='https://drive.google.com/uc?export=download&id=1BCxLPz0rAnVK-BUl7fYGMHTlsL-sMZoL'
  url2='https://drive.google.com/uc?export=download&id=1-8bSjFdd_2gWrL4rC9XEhityLQlyrrTn'



  output1 = '/content/SingleA.npy'
  output2 = '/content/SingleB.npy'
  gdown.download(url1, output1, quiet=True)
  gdown.download(url2, output2, quiet=True)



#######################################################
################# Dataset constructor #################
#######################################################

def MakeDataset(nrow,TempDimLen,nly,Dataset='single',mode='cls'):

  try:
    del(X_test,X_train,Y_test,Y_test)


  except:
    pass
  if not os.path.isfile('/content/SingleB.npy'):
    DownLoadData(DatasetType=Dataset)


  if Dataset=='single':
    SingleA=np.load('SingleA.npy')
    Xset,Yset=Map3D(SingleA,nrow,TempDimLen,nly,labels=None)
    del(SingleA)
    SingleB=np.load('SingleB.npy')
    Xset1,Yset1=Map3D(SingleB,nrow,TempDimLen,nly,labels=None)
    del(SingleB)

    Xset=np.vstack((Xset, Xset1))
    del(Xset1)
    Yset=np.vstack((Yset, Yset1))
    if mode=='mtl':
      Yset=Yset[:,1:]

  if Dataset=='double':
    Double=np.load('Double.npy')
    if mode=='mtl':
      damagedjoint=[[],[3,26],[7,14],[13,23],[21,25],[23,24]]# List of damaged joints in double damage cases
      Xset,Yset=Map3D(Double,nrow,TempDimLen,nly,labels=damagedjoint)
    else:
      Xset,Yset=Map3D(Double,nrow,TempDimLen,nly,labels=None)

  return Xset,Yset

Model constructor

In [ ]:
keras.utils.set_random_seed(42)
def build_conv3D_model(nout=31,fc_dim=32,Mode='cls',Shape=(6,5,512,1)):



  inputs = keras.layers.Input(shape=Shape)
  x= keras.layers.Conv3D(filters=2, kernel_size=(3,3,51),padding="same", name="Conv3D_1")(inputs)
  x= keras.layers.BatchNormalization()(x)
  x= keras.layers.ReLU()(x)
  x= keras.layers.Conv3D(filters=4, kernel_size=(3,3,21),padding="same", name="Conv3D_2")(x)
  x= keras.layers.BatchNormalization()(x)
  x= keras.layers.ReLU()(x)
  x= keras.layers.Conv3D(filters=8, kernel_size=(3,3,11),padding="same", name="Conv3D_3")(x)
  x= keras.layers.BatchNormalization()(x)
  x= keras.layers.ReLU()(x)
  x= keras.layers.Conv3D(filters=8, kernel_size=(3,3,7),padding="same", name="Conv3D_4")(x)
  x= keras.layers.BatchNormalization()(x)
  x= keras.layers.ReLU()(x)
  x= keras.layers.MaxPooling3D(pool_size=(1, 1, 16), name="MaxPooling3D-1")(x)

  x= keras.layers.Conv3D(filters=2, kernel_size=(3,3,11),padding="same", name="Conv3D_5")(x)
  x= keras.layers.BatchNormalization()(x)
  x= keras.layers.ReLU()(x)
  x= keras.layers.Conv3D(filters=4, kernel_size=(3,3,7),padding="same", name="Conv3D_6")(x)
  x= keras.layers.BatchNormalization()(x)
  x= keras.layers.ReLU()(x)
  x= keras.layers.Conv3D(filters=8, kernel_size=(3,3,5),padding="same", name="Conv3D_7")(x)
  x= keras.layers.BatchNormalization()(x)
  x= keras.layers.ReLU()(x)
  x= keras.layers.Conv3D(filters=8, kernel_size=(3,3,3),padding="same", name="Conv3D_8")(x)
  x= keras.layers.BatchNormalization()(x)
  x= keras.layers.ReLU()(x)
  x= keras.layers.MaxPooling3D(pool_size=(2, 2, 8), name="MaxPooling3D-2")(x)



  # flatten the features and feed into the Dense network
  x = keras.layers.Flatten(name="encode_flatten")(x)


  # we arbitrarily used fc_dim units here but feel free to change and see what results you get
  x =keras.layers.Dense(fc_dim, name="dense_1")(x)
  x= keras.layers.BatchNormalization()(x)
  x= keras.layers.ReLU()(x)


  x = keras.layers.Dense(fc_dim, name="Dense_2")(x)
  x= keras.layers.BatchNormalization()(x)
  x= keras.layers.ReLU()(x)

  if Mode=='cls':
    x=keras.layers.Dense(nout,activation="softmax", name="Dense_3")(x)#For Normal Classification
  elif Mode=='mtl':
    x=keras.layers.Dense(nout,activation="sigmoid", name="Dense_3")(x)#For Multi-task learing


  model=Model(inputs=inputs, outputs=x)
  return model

#CLS-Single

In this section classification is done for single damage cases

In [ ]:
TsetSize=0.2
Xset, Yset=MakeDataset(nrow=6,TempDimLen=512,nly=30,Dataset='single',mode='cls')
Xset=Xset[...,np.newaxis]
X_train, X_test, Y_train, Y_test = train_test_split(Xset, Yset, test_size=TsetSize)
del(Xset, Yset)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((25395, 6, 5, 512), (6349, 6, 5, 512), (25395, 31), (6349, 31))

For `X_train, X_test, Y_train, Y_test` You should get tensor with the following sizes:

((25395, 6, 5, 512), (6349, 6, 5, 512), (25395, 31), (6349, 31))

In [ ]:
nclas=31
CLS_SingleModel = build_conv3D_model(nout=31,fc_dim=8,Mode='cls')
CLS_SingleModel.summary()


In [ ]:
lrr=1e-3
EPOCHS = 30

step = tf.Variable(0, trainable=False)
boundaries = [10,20]
values = [lrr,lrr/10,lrr/100]
learning_rate_fn = keras.optimizers.schedules.PiecewiseConstantDecay(
boundaries, values)

optimizer = keras.optimizers.Adam(learning_rate =learning_rate_fn(step))#


CLS_SingleModel.compile(loss=keras.losses.CategoricalCrossentropy(),optimizer=optimizer,metrics='accuracy')# metrics='accuracy')keras.losses.SparseCategoricalCrossentropy()
# Store training stats
# Save model at each epoch
if not os.path.exists("Models"):
    os.mkdir("Models")
model_checkpoint_callback = ModelCheckpoint(
    filepath = 'Models/BestModel_cls_single.h5',
    monitor='val_loss',#'val_accuracy',
    mode='min',
    save_best_only=True,
    save_weights_only = False)

history = CLS_SingleModel.fit(X_train, Y_train, epochs=EPOCHS,batch_size=128, validation_data=( X_test, Y_test),
                     verbose=1,callbacks = [model_checkpoint_callback])#

In [ ]:
n_class=nclas
bestmodel = load_model('Models/BestModel_cls_single.h5')
y_pred_probs = bestmodel.predict(X_test)   # Softmax class probabilities from model
y_pred = np.argmax(y_pred_probs, axis = 1)
y_pred_oh = to_categorical(y_pred, n_class)

plot_performance(history,EPOCHS,'Learning Curve',save='/content/cls_single')
PlotConfusionMatrix( Y_test, y_pred_probs,figsize=(20,20),save='/content/cls_single')

# model scores
scores=score_model(Y_test, y_pred_oh, y_pred_probs,save = '/content/cls_single.csv')
# pred_confidence(y_pred_probs)
pd.DataFrame(scores,index=[0])

# Double Damage Cases Dataset Availability and Usage Instructions

The double damage cases dataset used in this experiment is not publicly available. We obtained access to this dataset through the QUGS lab, and we are grateful for their cooperation. Unfortunately, we are unable to provide the dataset here due to restrictions.

If you intend to reproduce the experiments related to the double damage cases, you would need to obtain the data yourself. To do this, you can request access to the double damage cases dataset from the individuals mentioned in http://www.structuralvibration.com/. If they grant access, they will provide you with a text file containing the data, not a Numpy array.

To integrate the dataset into the following experiments, please follow these steps:
1. Download the double damage cases text files
2. Download the one undamaged text file from http://www.structuralvibration.com/benchmark/download/.
3. Place these downloaded files into a directory.
4. Rename the files as 'zzzDD0.TXT', 'zzzDD1.TXT', ..., 'zzzDD5.TXT'.

'zzzDD0.TXT' refers to undamage case

After preparing the data as mentioned above, you can utilize the provided code snippets to preprocess the dataset for your experiments. The code will load the text files, process them, and concatenate them into a format suitable for further analysis.

```python
# Replace 'directory' with the actual path to the directory containing the text files.
directory = '/text_files_directory/'

for i in range(0, 6):
    respmat = np.loadtxt(directory + 'zzzDD' + str(i) + '.TXT', skiprows=11)
    respmat2 = np.delete(respmat, 0, 1).T[np.newaxis, ...]

    try:
        Double = np.concatenate((Double, respmat2), axis=0)
        del(respmat, respmat2)
    except:
        Double = respmat2
        del(respmat, respmat2)

print('Dataset shape: ', Double.shape)
np.save('Double.npy', Double)
```

Please ensure that the dataset shape matches the expected shape of (6, 30, 262144), and that the file 'Double.npy' is saved in the current directory.

Once the dataset is prepared using the provided code, you can proceed to run the subsequent sections of the experiment.


#CLS-Double

In [ ]:
try:
  del(X_train, X_test, Y_train, Y_test)
except:
  pass
TsetSize=0.2
Xset, Yset=MakeDataset(nrow=6,TempDimLen=512,nly=30,Dataset='double',mode='cls')
X_train, X_test, Y_train, Y_test = train_test_split(Xset, Yset, test_size=TsetSize)
del(Xset, Yset)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

For `X_train, X_test, Y_train, Y_test` You should get tensor with the following sizes:

((2457, 6, 5, 512), (615, 6, 5, 512), (2457, 6), (615, 6))

In [ ]:
nclas=6 # 5 double damage cases plus one undamaged case
CLS_DoubleModel = build_conv3D_model(nout=nclas,fc_dim=8,Mode='cls')
CLS_DoubleModel.summary()


In [ ]:
lrr=1e-3
EPOCHS = 30

step = tf.Variable(0, trainable=False)
boundaries = [10,20]
values = [lrr,lrr/10,lrr/100]
learning_rate_fn = keras.optimizers.schedules.PiecewiseConstantDecay(
boundaries, values)

optimizer = keras.optimizers.Adam(learning_rate =learning_rate_fn(step))


CLS_DoubleModel.compile(loss=keras.losses.CategoricalCrossentropy(),optimizer=optimizer,metrics='accuracy')#
# Store training stats
# Save model at each epoch
if not os.path.exists("Models"):
    os.mkdir("Models")
model_checkpoint_callback = ModelCheckpoint(
    filepath = 'Models/BestModel_cls_double.h5',
    monitor='val_accuracy',#'val_accuracy',
    mode='max',
    save_best_only=True,
    save_weights_only = False)

history = CLS_DoubleModel.fit(X_train, Y_train, epochs=EPOCHS,batch_size=128, validation_data=( X_test, Y_test),
                     verbose=1,callbacks = [model_checkpoint_callback])#

In [ ]:
n_class=nclas
bestmodel = load_model('Models/BestModel_cls_double.h5')
y_pred_probs = bestmodel.predict(X_test)   # Softmax class probabilities from model
y_pred = np.argmax(y_pred_probs, axis = 1)
y_pred_oh = to_categorical(y_pred, n_class)

plot_performance(history,EPOCHS,'Learning Curve',save='/content/cls_double')
PlotConfusionMatrix( Y_test, y_pred_probs,figsize=(8,8),save='/content/cls_double')

# model scores
scores=score_model(Y_test, y_pred_oh, y_pred_probs,save = '/content/cls_double.csv')
# pred_confidence(y_pred_probs)
pd.DataFrame(scores,index=[0])

#CLS-Double-TL


In [ ]:
try:
  del(X_train, X_test, Y_train, Y_test)
except:
  pass
TsetSize=0.2
Xset, Yset=MakeDataset(nrow=6,TempDimLen=512,nly=30,Dataset='double',mode='cls')
X_train, X_test, Y_train, Y_test = train_test_split(Xset, Yset, test_size=TsetSize)
del(Xset, Yset)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

For `X_train, X_test, Y_train, Y_test` You should get tensor with the following sizes:

((2457, 6, 5, 512), (615, 6, 5, 512), (2457, 6), (615, 6))

In [ ]:
#Note The besed save model should be provided in the for below line in the specified directory
BestModel_cls_single = load_model('Models/BestModel_cls_single.h5')#.layers[:-6].output
xx = keras.layers.Dense(6,activation='softmax')(BestModel_cls_single.layers[-2].output)
CLS_Double_TL_Model=Model(inputs=BestModel_cls_single.input,outputs=xx)

for layer in CLS_Double_TL_Model.layers[:-7]:
  layer.trainable = False

CLS_Double_TL_Model.summary()

In [ ]:
lrr=1e-3
EPOCHS = 30

step = tf.Variable(0, trainable=False)
boundaries = [10,20]
values = [lrr,lrr/10,lrr/100]
learning_rate_fn = keras.optimizers.schedules.PiecewiseConstantDecay(
boundaries, values)

optimizer = keras.optimizers.Adam(learning_rate =learning_rate_fn(step))


CLS_Double_TL_Model.compile(loss=keras.losses.CategoricalCrossentropy(),optimizer=optimizer,metrics='accuracy')# metrics='accuracy')keras.losses.SparseCategoricalCrossentropy()
# Store training stats
# Save model at each epoch
if not os.path.exists("Models"):
    os.mkdir("Models")
model_checkpoint_callback = ModelCheckpoint(
    filepath = 'Models/BestModel_cls_double(TL).h5',
    monitor='val_accuracy',#'val_accuracy',
    mode='max',
    save_best_only=True,
    save_weights_only = False)

history = CLS_Double_TL_Model.fit(X_train, Y_train, epochs=EPOCHS,batch_size=128, validation_data=( X_test, Y_test),
                     verbose=1,callbacks = [model_checkpoint_callback])#

In [ ]:
n_class=6
bestmodel = load_model('Models/BestModel_cls_double(TL).h5')
y_pred_probs = bestmodel.predict(X_test)   # Softmax class probabilities from model
y_pred = np.argmax(y_pred_probs, axis = 1)
y_pred_oh = to_categorical(y_pred, n_class)

plot_performance(history,EPOCHS,'Learning Curve',save='/content/cls_double(TL)')
PlotConfusionMatrix( Y_test, y_pred_probs,figsize=(8,8),save='/content/cls_double(TL)')

# model scores
scores=score_model(Y_test, y_pred_oh, y_pred_probs,save = '/content/cls_double(TL).csv')
# pred_confidence(y_pred_probs)
pd.DataFrame(scores,index=[0])

#Multi-Task Learning

In [ ]:
try:
  del(X_train, X_test, Y_train, Y_test)
except:
  pass

TsetSize=0.2
Xset, Yset=MakeDataset(nrow=6,TempDimLen=512,nly=30,Dataset='single',mode='mtl')
Xset1, Yset1=MakeDataset(nrow=6,TempDimLen=512,nly=30,Dataset='double',mode='mtl')
Xset1, Yset1= Xset1[512:], Yset1[512:]
Xset=np.vstack((Xset,Xset1))
Yset=np.vstack((Yset,Yset1))
del(Xset1, Yset1)

X_train, X_test, Y_train, Y_test = train_test_split(Xset, Yset, test_size=TsetSize)
del(Xset, Yset)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

For `X_train, X_test, Y_train, Y_test` You should get tensor with the following sizes:

((27443, 6, 5, 512), (6861, 6, 5, 512), (27443, 30), (6861, 30))

In [ ]:
# Count the number of 1's in each sample of Yset
n_ones = np.sum(Y_test, axis=1)

# Separate samples with one 1 and two 1's
one_1_mask = (n_ones == 1)
two_1_mask = (n_ones == 2)


In [ ]:
ntask=30
MTL_Model = build_conv3D_model(nout=ntask,fc_dim=8,Mode='mtl')
MTL_Model.summary()

In [ ]:
#This class Allows us to evaluate the model for multiple test sets

class AdditionalValidationSets(tf.keras.callbacks.Callback):
    def __init__(self, validation_sets1,validation_sets2, verbose=0, batch_size=None):
        """
        :param validation_sets:
        a list of 3-tuples (validation_data, validation_targets, validation_set_name)
        or 4-tuples (validation_data, validation_targets, sample_weights, validation_set_name)
        :param verbose:
        verbosity mode, 1 or 0
        :param batch_size:
        batch size to be used when evaluating on the additional datasets
        """
        super(AdditionalValidationSets, self).__init__()
        self.validation_sets1 = validation_sets1
        for validation_set in self.validation_sets1:
            if len(validation_set) not in [3, 4]:
                raise ValueError()

        self.validation_sets2 = validation_sets2
        for validation_set in self.validation_sets2:
            if len(validation_set) not in [3, 4]:
                raise ValueError()

        self.epoch = []
        self.history = {}
        self.verbose = verbose
        self.batch_size = batch_size

    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)

        # record the same values as History() as well
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        # evaluate on the additional validation sets
        for validation_set in self.validation_sets1:
            if len(validation_set) == 3:
                validation_data, validation_targets, validation_set_name = validation_set
                sample_weights = None
            elif len(validation_set) == 4:
                validation_data, validation_targets, sample_weights, validation_set_name = validation_set
            else:
                raise ValueError()

            results = self.model.evaluate(x=validation_data,
                                          y=validation_targets,
                                          verbose=self.verbose,
                                          sample_weight=sample_weights,
                                          batch_size=self.batch_size)

            for metric, result in zip(self.model.metrics_names,results):
                valuename = validation_set_name + '_' + metric
                self.history.setdefault(valuename, []).append(result)

        for validation_set in self.validation_sets2:
            if len(validation_set) == 3:
                validation_data, validation_targets, validation_set_name = validation_set
                sample_weights = None
            elif len(validation_set) == 4:
                validation_data, validation_targets, sample_weights, validation_set_name = validation_set
            else:
                raise ValueError()

            results = self.model.evaluate(x=validation_data,
                                          y=validation_targets,
                                          verbose=self.verbose,
                                          sample_weight=sample_weights,
                                          batch_size=self.batch_size)

            for metric, result in zip(self.model.metrics_names,results):
                valuename = validation_set_name + '_' + metric
                self.history.setdefault(valuename, []).append(result)

In [ ]:
lrr=1e-3
EPOCHS = 10

step = tf.Variable(0, trainable=False)
boundaries = [10,20]
values = [lrr,lrr/10,lrr/100]
learning_rate_fn = keras.optimizers.schedules.PiecewiseConstantDecay(
boundaries, values)

def MT_accuracy(y_true, y_pred):
    y_true = tf.cast(tf.round(y_true), dtype=tf.float32)
    y_pred = tf.cast(tf.round(y_pred), dtype=tf.float32)
    ns=tf.shape(y_true)[0]
    count=0
    for i in range(ns):
      if tf.experimental.numpy.array_equal( y_true[i,:], y_pred[i,:]):
        count+=1
    return count/ns

optimizer = keras.optimizers.Adam(learning_rate =learning_rate_fn(step))

MTL_Model.compile(loss=keras.losses.BinaryCrossentropy(),optimizer=optimizer,metrics=[MT_accuracy])# metrics='accuracy')keras.losses.SparseCategoricalCrossentropy()
# Store training stats
# Save model at each epoch
if not os.path.exists("Models"):
    os.mkdir("Models")
model_checkpoint_callback = ModelCheckpoint(
    filepath = 'Models/BestMTL_Model.h5',
    monitor='val_MT_accuracy',#'val_accuracy',
    mode='max',
    save_best_only=True,
    save_weights_only = False)

HIST = AdditionalValidationSets([(X_test[one_1_mask],Y_test[one_1_mask], 'val: Single Damage')], [(X_test[two_1_mask],Y_test[two_1_mask], 'val: Double Damage')])

MTL_Model.fit(X_train, Y_train, epochs=EPOCHS,batch_size=64, validation_data=( X_test, Y_test),
                     verbose=1,callbacks = [model_checkpoint_callback,HIST])#

In [ ]:
bestmodel = load_model('Models/BestMTL_Model.h5',custom_objects={"MT_accuracy": MT_accuracy})
y_pred_probs = bestmodel.predict(X_test)   # Softmax class probabilities from model
plot_performance(HIST,EPOCHS,'Learning Curve',save='/content/MTL')

In [ ]:
y_pred = np.round(y_pred_probs).astype(int)
# Calculate task-specific accuracy
MT_eval(y_pred,Y_test,30,save='/content/MTL')
